In [114]:
import numpy as np
from scipy import stats
import pandas as pd

**1)** Давайте вернёмся к данным выживаемости пациентов с лейкоцитарной лимфомой из видео про критерий знаков:

```49, 58, 75, 110, 112, 132, 151, 276, 281, 362*```

Измерено остаточное время жизни с момента начала наблюдения (в неделях); звёздочка обозначает цензурирование сверху — исследование длилось 7 лет, и остаточное время жизни одного пациента, который дожил до конца наблюдения, неизвестно.

Поскольку цензурировано только одно наблюдение, для проверки гипотезы ```H_0: medX = 200``` на этих данных можно использовать критерий знаковых рангов — можно считать, что время дожития последнего пациента в точности равно 362, на ранг этого наблюдения это никак не повлияет. 

**Критерием знаковых рангов** проверьте эту гипотезу против двусторонней альтернативы, введите достигаемый уровень значимости, округлённый до четырёх знаков после десятичной точки.

$H_0\colon$ медиана остаточного время жизни равна 200

$H_1\colon$ медиана остаточного время жизни не равна 200

In [117]:
# Описываем выборку
sample = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])
medX = 200

# Критерий Знаковых Рангов Уилкоксона
print('Wilcoxon Criterion\nStatistics: %.1f, p-value: ', stats.wilcoxon(sample - medX))

Wilcoxon Criterion
Statistics: %.1f, p-value:  WilcoxonResult(statistic=17.0, pvalue=0.322265625)


Нулевая гипотеза не отвергается, мы не модем сказать что медиана остаточного время жизни не равна 200. Возможно это так.

**2)** В ходе исследования влияния лесозаготовки на биоразнообразие лесов острова Борнео собраны данные о количестве видов деревьев в ```12``` лесах, где вырубка не ведётся: ```22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15``` и в 9 лесах, где идёт вырубка: ```17, 18, 18, 15, 12, 4, 14, 15, 10```

Проверьте гипотезу о равенстве среднего количества видов в двух типах лесов против односторонней альтернативы о снижении биоразнообразия в вырубаемых лесах. Используйте ранговый критерий. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки.

$H_0\colon$ среднее количество видов в лесах с вырубкой больше

$H_1\colon$ среднее количество видов в лесах с вырубкой снижается

Наши выборки независимые, следовательно нужно использовать двухвыборочные ранговые критерии для независимых выборок. В нашем случае это критерий Манна - Уитни:

In [27]:
# Опишем наши выборки 
sample_no_felling = np.array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])
sample_felling = np.array([17, 18, 18, 15, 12, 4, 14, 15, 10])

stats.mannwhitneyu(sample_no_felling, sample_felling, alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.02900499272087373)

Нулевая гипотеза отвергается, среднее количество видов в лесах с вырубкой снижается.

**3)** 28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны **температура воздуха** и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для **разности средних температур воздуха** при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:
- установите ```np.random.seed(0)``` перед первым вызовом функции ```get_bootstrap_samples()```, один раз
- сделайте по **1000** псевдовыборок из каждой выборки.

In [79]:
# Загрузка данных
data = pd.read_csv('challenger.txt', sep = '\t')
data.columns = ['Date', 'Temperature', 'Incident']
data.head()

,Date,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [100]:
# Отбираем группы (берем именно DataFrame, т.к. функция bootstrap завязана на нем)
sample_incident = data[data['Incident'] == 1][['Temperature']]
sample_no_incident = data[data['Incident'] == 0][['Temperature']]

print('Mean Temperature (Incident): %.4f' %sample_incident.mean()[0])
print('Mean Temperature (No Incident): %.4f' %sample_no_incident.mean()[0])

Mean Temperature (Incident): 17.6143
Mean Temperature (No Incident): 22.2812


In [74]:
# Функции для ДИ на основе бутстрепа

# Создадим функцию для получения псевдоподвыборок на основе bootstrap
def get_bootstrap_samples(data, n_samples):
    """
    n_samples - число выборок 
    data - исходная выборка (np.array)
    
    """
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

# Функция для расчета статистик (квантиль)
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2, 100 * (1 - alpha / 2)])
    return boundaries

In [93]:
np.random.seed(0)

# Создаем бутстреп псевдовыборки. В каждой выборке оцениваем среднее.
# Получаем средние по всем нагенерированным псевдовыборкам
incidents_means = list(map(np.mean, get_bootstrap_samples(sample_incident['Temperature'].values, n_samples = 1000)))
no_incidents_means = list(map(np.mean, get_bootstrap_samples(sample_no_incident['Temperature'].values, n_samples = 1000)))

# Рассчитываем ДИ
conf_int = stat_intervals(np.array(incidents_means) - np.array(no_incidents_means), alpha = 0.05)
print(f'95% Confidence Interval: [{round(conf_int[0], 4)}, {round(conf_int[1], 4)}]')

95% Confidence Interval: [-8.0646, -1.4504]


Видно, что выборки значимо отличаются друг от друга. ДИ смещен в лево и не включает ноль.

**4)** На данных предыдущей задачи проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? Округлите до четырёх знаков после десятичной точки. 

Чтобы получить такое же значение, как мы:
- установите ```np.random.seed(0)```
- возьмите ```10000``` перестановок

Выборки будут независимыми,т.к. в одни дни кольца повреждались, а в други нет. Воспользуемся готовой реализацией

$H_0\colon$ средняя температура воздуха одинакова когда уплотнительные кольца повреждаются и нет

$H_1\colon$ средняя температура воздуха отличается когда уплотнительные кольца повреждаются и нет

In [109]:
# Расчет t - статистики для независимых выборок
def permutation_t_stat_ind(sample_1, sample_2):
    return sample_1.mean() - sample_2.mean()

# Функция генерирования индексов для разбиения данных на первую и вторую выборку случайным образом
def get_random_combinations(n1, n2, max_combinations):
    
    index = np.arange(n1 + n2)
    indices = set([tuple(index)])
    
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
        
    return [(index[:n1], index[n1:]) for index in indices]

def get_permutation_zero_distr_ind(sample_1, sample_2, max_combinations = None):
    # Объединим выборки в одну и рассчитаем необходимые параметры
    joined_sample = np.hstack((sample_1, sample_2))
    
    n_1 = len(sample_1)
    n_2 = len(sample_2)
    n = len(joined_sample)
    
    # Если нам задано ограничение на максимальное количество комбинаций, 
    # то сгенерируем индексы для разбиения данных на первую и вторую выборку случайным образом
    if max_combinations:
        indices = get_random_combinations(n_1, n_2, max_combinations)
        
    # Иначе явно перебираем все возможные комбинации 
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n_1)]
        
    # построим разделение нашей выборки на две по соответствующим индексам 
    # Далее рассчитаем нужную статистику, то есть разницу средних по полученным разбиениям
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    
    return distr

# Получение p-value
def get_p_value_permutation_test_2sample_ind(sample_1, sample_2, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError ('Alternative Not Recognized!!!')
        
    t_stat = round(permutation_t_stat_ind(sample_1, sample_2), 4)
    zero_distribution = get_permutation_zero_distr_ind(sample_1, sample_2, max_permutations)
    
    if alternative == 'two-sided':
        return t_stat, round(sum([1 if abs(x) >= abs(t_stat) else 0 for x in zero_distribution]) / len(zero_distribution), 4)
    
    if alternative == 'less':
        return t_stat, round(sum([1 if x <= t_stat else 0 for x in zero_distribution]) / len(zero_distribution), 4)

    if alternative == 'greater':
        return t_stat, round(sum([1 if x >= t_stat else 0 for x in zero_distribution]) / len(zero_distribution), 4)

In [113]:
np.random.seed(0)

# p-value для 10к перестановок
test_res_1 = get_p_value_permutation_test_2sample_ind(
    sample_1 = sample_incident['Temperature'].values,
    sample_2 = sample_no_incident['Temperature'].values,
    max_permutations = 10000,
    alternative = 'two-sided'
)
print(f'Statistics: {test_res_1[0]}, p-value: {test_res_1[1]}')

Statistics: -4.667, p-value: 0.0054


Нулевая гипотеза отвергается. Обнаружены значимые различия в температуре воздуха когда кольца повреждались и нет.